# Hypothesis testing: Completion Rate
The goal is to determine if the difference in completion rates between the Test (new design) and Control (old design) groups is statistically significant. This will help confirm if the redesign has had a meaningful impact on the completion rate.

## Completion Hypothesis:
#### Null hypothesis (H₀): The new design does not lead to a higher completion rate than the old design.
#### Alternative hypothesis (H₁): The new design leads to a higher completion rate than the old design.

In [1]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
import scipy.stats as st
from scipy.stats import norm

In [2]:
control = pd.read_csv('../data/clean/merged_control.csv')
control = control.drop(columns = 'Unnamed: 0')
display(control)

test = pd.read_csv('../data/clean/merged_test.csv')
test = test.drop(columns = 'Unnamed: 0')
display(test)

,client_id,visitor_id,visit_id,process_step,date_time
0,1104,194240915_18158000533,543158812_46395476577_767725,start,2017-06-12 07:49:18
1,1104,194240915_18158000533,643221571_99977972121_69283,start,2017-06-20 22:31:33
2,1186,446844663_31615102958,507052512_11309370126_442139,start,2017-04-08 15:59:16
3,1186,446844663_31615102958,795373564_99931517312_810896,start,2017-04-08 18:05:02
4,1186,446844663_31615102958,795373564_99931517312_810896,step_1,2017-04-08 18:05:13
...,...,...,...,...,...
103401,9997391,494669706_3354361161,84654768_90613632047_633963,step_2,2017-04-05 15:41:34
103402,9997391,494669706_3354361161,84654768_90613632047_633963,step_3,2017-04-05 15:41:39
103403,9997470,395791369_55562604618,904791598_9725982898_416914,start,2017-04-20 20:04:38
103404,9997470,91394485_75296404278,655572400_94971272893_411965,start,2017-04-07 16:11:03


,client_id,visitor_id,visit_id,process_step,date_time
0,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56
1,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03
2,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35
3,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14
4,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34
...,...,...,...,...,...
128686,9999729,843385170_36953471821,493310979_9209676464_421146,start,2017-04-20 14:21:27
128687,9999729,843385170_36953471821,493310979_9209676464_421146,step_1,2017-04-20 14:22:49
128688,9999729,843385170_36953471821,493310979_9209676464_421146,step_2,2017-04-20 14:27:36
128689,9999832,145538019_54444341400,472154369_16714624241_585315,start,2017-05-16 16:46:03


## First, let's test the differences between visit_ids

In [3]:
n_control = control['visit_id'].nunique()
n_test = test['visit_id'].nunique()

# Identify visits who completed the whole process
completed_visits_control = control[control['process_step'] == 'confirm']['visit_id'].nunique()
completed_visits_test = test[test['process_step'] == 'confirm']['visit_id'].nunique()

print(f"Total number of visits in the control group: {n_control}")
print(f"Visits who completed the process in the control group: {completed_visits_control}\n")
print(f"Total number of visits in the test group: {n_test}")
print(f"Visits who completed the process in the test group: {completed_visits_test}")

Total number of visits in the control group: 28527
Visits who completed the process in the control group: 13176

Total number of visits in the test group: 30799
Visits who completed the process in the test group: 16593


In [4]:
# Calculate proportions
p_control = completed_visits_control / n_control
p_test = completed_visits_test / n_test

display(p_control)
display(p_test)

0.4618782206330844

0.5387512581577324

In [5]:
# Prepare the data for the Z-test
count = np.array([completed_visits_control, completed_visits_test])  # Success counts
nobs = np.array([n_control, n_test])  # Total observations

# Perform Two-proportion Z-test
z_stat, p_value = sm.stats.proportions_ztest(count, nobs)

# Print the results
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret the results
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in completion rates.")
else:
    print("Fail to reject the null hypothesis: No significant difference in completion rates.")

Z-statistic: -18.7103
P-value: 0.0000
Reject the null hypothesis: There is a significant difference in completion rates.


## Now, let's try the differences between client_ids

In [6]:
nc_control = control['client_id'].nunique()
nc_test = test['client_id'].nunique()

# Identify clients who completed the whole process
completed_clients_control = control[control['process_step'] == 'confirm']['client_id'].nunique()
completed_clients_test = test[test['process_step'] == 'confirm']['client_id'].nunique()

print(f"Total number of clients in the control group: {nc_control}")
print(f"Clients who completed the process in the control group: {completed_clients_control}\n")
print(f"Total number of clients in the test group: {nc_test}")
print(f"Clients who completed the process in the test group: {completed_clients_test}")

Total number of clients in the control group: 21745
Clients who completed the process in the control group: 13161

Total number of clients in the test group: 25145
Clients who completed the process in the test group: 16565


In [7]:
# Calculate proportions
pc_control = completed_clients_control / nc_control
pc_test = completed_clients_test / nc_test

display(pc_control)
display(pc_test)

0.6052425845021844

0.6587790813282959

In [8]:
# Prepare the data for the Z-test
count = np.array([completed_clients_control, completed_clients_test])  # Success counts
nobs = np.array([nc_control, nc_test])  # Total observations

# Perform Two-proportion Z-test
z_stat, p_value = sm.stats.proportions_ztest(count, nobs)

# Print the results
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret the results
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in completion rates.")
else:
    print("Fail to reject the null hypothesis: No significant difference in completion rates.")

Z-statistic: -12.0010
P-value: 0.0000
Reject the null hypothesis: There is a significant difference in completion rates.
